In [1]:
from ibl_pipeline.analyses import behavior
from ibl_pipeline import behavior as behavior_ingest
from ibl_pipeline import subject, acquisition
from ibl_pipeline.utils import psychofit as psy
import plotting_utils as putils
from uuid import UUID
import numpy as np
import datetime
import datajoint as dj
import json
import seaborn as sns
import pandas as pd
import plotly
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)

Connecting shan@datajoint.internationalbrainlab.org:3306


In [2]:
subj = subject.Subject & {'subject_nickname': 'CSHL_007'}
# get trial counts and session length to date
session_info = (behavior.BehavioralSummaryByDate * 
                behavior.BehavioralSummaryByDate.ReactionTimeByDate & 
                subj).proj('session_date', 'performance_easy', 'median_reaction_time').fetch(as_dict=True)
session_info = pd.DataFrame(session_info)

# get the first date when animal became "trained" and "ready for ephys"
status = putils.get_status(subj)

# get date range and mondays
d = putils.get_date_range(subj)

In [25]:
yrange = [0, 1.1]
performance_easy = go.Scatter(
    x=[t.strftime('%Y-%m-%d') for t in session_info['session_date'].tolist()],
    y=session_info['performance_easy'].tolist(),
    mode='markers+lines',
    marker=dict(
        size=6,
        color='black',
        line=dict(
            color='white',
            width=1
        )
    ),
    name='performance easy',
    yaxis='y1',
    showlegend=False
)
rt = go.Scatter(
    x=[t.strftime('%Y-%m-%d') for t in session_info['session_date'].tolist()],
    y=session_info['median_reaction_time'].tolist(),
    mode='markers+lines',
    marker=dict(
        size=6,
        color='red',
        line=dict(
            color='white',
            width=1)
    ),
    name='reaction time',
    yaxis='y2',
    showlegend=False
)

data = [performance_easy, rt]

# add monday plots
data = putils.create_monday_plot(data, yrange, d['mondays'])

# add status plots
data = putils.create_status_plot(data, yrange, status)
    
layout = go.Layout(

    yaxis=dict(
        title='Performance on easy trials',
        range=yrange
    ),
    yaxis2=dict(
        title='Median reaction time (s)',
        overlaying='y',
        color = 'red',
        side='right',
        type='log',
        range=np.log10([0.1, 10]).tolist(),
        dtick=np.log10([0.1, 1, 10]).tolist()
        
    ),
    xaxis=dict(
        title='Date',
        showgrid=False
    ), 
    width=700,
    height=400,
    title=dict(
        text='Performance and median reaction time',
        x=0.14,
        y=0.85
    ),
    legend=dict(
        x=1.2, 
        y=0.8,
        orientation='v')
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [15]:
f = open("performance_reaction_time.json","w")
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()